# Hello Rwanda
### Predict CO2 Emissions in Rwanda


Competition Website : https://www.kaggle.com/competitions/playground-series-s3e20/overview

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Loading Data

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s3e20/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e20/test.csv')
ss = pd.read_csv('/kaggle/input/playground-series-s3e20/sample_submission.csv')

train.shape, test.shape, ss.shape

((79023, 76), (24353, 75), (24353, 2))

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79023 entries, 0 to 79022
Data columns (total 76 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   ID_LAT_LON_YEAR_WEEK                                      79023 non-null  object 
 1   latitude                                                  79023 non-null  float64
 2   longitude                                                 79023 non-null  float64
 3   year                                                      79023 non-null  int64  
 4   week_no                                                   79023 non-null  int64  
 5   SulphurDioxide_SO2_column_number_density                  64414 non-null  float64
 6   SulphurDioxide_SO2_column_number_density_amf              64414 non-null  float64
 7   SulphurDioxide_SO2_slant_column_number_density            64414 non-null  float64
 8   SulphurDioxide_c

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24353 entries, 0 to 24352
Data columns (total 75 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   ID_LAT_LON_YEAR_WEEK                                      24353 non-null  object 
 1   latitude                                                  24353 non-null  float64
 2   longitude                                                 24353 non-null  float64
 3   year                                                      24353 non-null  int64  
 4   week_no                                                   24353 non-null  int64  
 5   SulphurDioxide_SO2_column_number_density                  18211 non-null  float64
 6   SulphurDioxide_SO2_column_number_density_amf              18211 non-null  float64
 7   SulphurDioxide_SO2_slant_column_number_density            18211 non-null  float64
 8   SulphurDioxide_c

In [5]:
nan_cols = [ col for col in train.columns if train[col].isnull().sum() > 0]
train[nan_cols]

,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,SulphurDioxide_sensor_zenith_angle,SulphurDioxide_solar_azimuth_angle,SulphurDioxide_solar_zenith_angle,SulphurDioxide_SO2_column_number_density_15km,CarbonMonoxide_CO_column_number_density,...,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle
0,-0.000108,0.603019,-0.000065,0.255668,-98.593887,50.843559,-130.050797,35.874496,-0.000027,0.035370,...,53534.732422,3664.436218,61085.809570,2615.120483,15.568533,0.272292,-12.628986,35.632416,-138.786423,30.752140
1,0.000021,0.728214,0.000014,0.130988,16.592861,39.137194,-140.874435,28.965133,0.000012,0.036526,...,63790.296241,3651.190311,66969.478735,3174.572424,8.690601,0.256830,30.359375,39.557633,-145.183930,27.251779
2,0.000514,0.748199,0.000385,0.110018,72.795837,52.868816,-150.191757,23.206415,0.000154,0.035338,...,55923.790554,4216.986492,60068.894448,3516.282669,21.103410,0.251101,15.377883,30.401823,-142.519545,26.193296
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.036790,...,44569.130636,5228.507736,51064.547339,4180.973322,15.386899,0.262043,-11.293399,24.380357,-132.665828,28.829155
4,-0.000079,0.676296,-0.000048,0.121164,4.121269,35.515587,-137.409159,24.331972,-0.000028,0.034675,...,59904.314844,3980.598120,63751.125781,3355.710107,8.114694,0.235847,38.532263,37.392979,-141.509805,22.204612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79018,0.000284,1.195643,0.000340,0.191313,72.820518,55.988022,-140.821274,25.148563,0.000131,0.026488,...,54703.511452,5459.185355,60657.101913,4590.879504,20.245954,0.304797,-35.140368,40.113533,-129.935508,32.095214
79019,0.000083,1.130868,0.000063,0.177222,-12.856753,19.435339,-131.114411,31.197638,0.000030,0.026049,...,53983.493958,5606.449457,60168.191528,4659.130378,6.104610,0.314015,4.667058,47.528435,-134.252871,30.771469
79020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.030098,...,49904.738442,6222.646776,56596.027209,5222.646823,14.817885,0.288058,-0.340922,35.328098,-134.731723,30.716166
79021,-0.000034,0.879397,-0.000028,0.184209,-100.344827,32.599393,-129.573396,33.906037,-0.000015,0.031012,...,41395.313100,7896.456885,46533.348194,6946.858022,32.594768,0.274047,8.427699,48.295652,-139.447849,29.112868


In [6]:
nan_cols = [ col for col in test.columns if test[col].isnull().sum() > 0]
test[nan_cols]

,SulphurDioxide_SO2_column_number_density,SulphurDioxide_SO2_column_number_density_amf,SulphurDioxide_SO2_slant_column_number_density,SulphurDioxide_cloud_fraction,SulphurDioxide_sensor_azimuth_angle,SulphurDioxide_sensor_zenith_angle,SulphurDioxide_solar_azimuth_angle,SulphurDioxide_solar_zenith_angle,SulphurDioxide_SO2_column_number_density_15km,CarbonMonoxide_CO_column_number_density,...,Cloud_cloud_top_pressure,Cloud_cloud_top_height,Cloud_cloud_base_pressure,Cloud_cloud_base_height,Cloud_cloud_optical_depth,Cloud_surface_albedo,Cloud_sensor_azimuth_angle,Cloud_sensor_zenith_angle,Cloud_solar_azimuth_angle,Cloud_solar_zenith_angle
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,36022.027344,8472.313477,41047.937500,7472.313477,7.935617,0.240773,-100.113792,33.697044,-133.047546,33.779583
1,0.000456,0.691164,0.000316,0.000000,76.239196,15.600607,-140.529848,28.896124,0.000157,0.037641,...,48539.737242,6476.147323,54915.708579,5476.147161,11.448437,0.293119,-30.510319,42.402593,-138.632822,31.012380
2,0.000161,0.605107,0.000106,0.079870,-42.055341,39.889060,-136.908976,30.054682,0.000053,0.037795,...,34133.080469,8984.795703,39006.093750,7984.795703,10.753179,0.267130,39.087361,45.936480,-144.784988,26.743361
3,0.000350,0.696917,0.000243,0.201028,72.169566,58.862543,-152.999440,21.806625,0.000093,0.039743,...,50854.991076,6014.724059,57646.368368,5014.724115,11.764556,0.304679,-24.465127,42.140419,-135.027891,29.604774
4,-0.000317,0.580527,-0.000184,0.204352,76.190865,15.646016,-135.794754,24.328299,-0.000077,0.039232,...,46594.685145,6849.280477,52896.541873,5849.280394,13.065317,0.284221,-12.907850,30.122641,-135.500119,26.276807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24348,-0.000618,0.745549,-0.000461,0.234492,72.306198,61.114494,-123.965439,19.866062,-0.000171,0.034566,...,48839.430415,6260.120033,55483.459980,5260.120056,30.398508,0.180046,-25.528588,45.284576,-116.521412,29.992562
24349,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.038438,...,47042.694849,6678.843299,53589.917383,5678.951521,19.223844,0.177833,-13.380005,43.770351,-122.405759,29.017975
24350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.033042,...,55337.148173,5336.282475,62646.761340,4336.282491,13.801194,0.219471,-5.072065,33.226455,-124.530639,30.187472
24351,0.000071,1.003805,0.000077,0.205077,74.327427,38.215228,-131.809054,26.745795,0.000037,0.030662,...,44813.691428,7188.578533,50728.313991,6188.578464,27.887489,0.247275,-0.668714,45.885617,-129.006797,30.427455


# Feature Eng.

In [7]:
#this part was taken from https://www.kaggle.com/code/johnsmith44/ps3e20-co2-emissions-in-rwanda-compact-trick
train_test = [train, test]

y = train['emission']

for idx, df in enumerate(train_test):
    # Adding 'month_no' column
    df['date'] = pd.to_datetime('2021' + df['week_no'].astype(str) + '0', format='%Y%W%w')
    df['month_no'] = df['date'].dt.month
    df.drop(columns=['date'], inplace=True)
    
    # Adding 'corona' active period column    
    df['corona'] = (df.year == 2020) & (df.month_no > 2)
    
    # Keeping 'latitude', 'longitude', 'week_no', 'month_no', 'corona' features only.
    train_test[idx] = df[['latitude', 'longitude', 'week_no', 'month_no', 'corona']]

train, test = train_test

In [8]:
train.shape

(79023, 5)

In [9]:
train.head()

,latitude,longitude,week_no,month_no,corona
0,-0.51,29.29,0,1,False
1,-0.51,29.29,1,1,False
2,-0.51,29.29,2,1,False
3,-0.51,29.29,3,1,False
4,-0.51,29.29,4,1,False


# Modeling

In [10]:
X_train, X_test, y_train, y_test = train_test_split(train, y, shuffle=True, test_size=0.2, random_state=123)

In [11]:
base_model = RandomForestRegressor(n_estimators=300, random_state=42)
ensemble_model = BaggingRegressor(base_estimator=base_model, n_estimators=5, random_state=42)
ensemble_model.fit(X_train, y_train)

y_pred_ensemble = ensemble_model.predict(X_test)
rmse_ensemble = np.sqrt(mean_squared_error(y_test, y_pred_ensemble))
print("Ensemble RMSE:", rmse_ensemble)

Ensemble RMSE: 14.15027344217334


### XGBRegressor

In [12]:
from xgboost import XGBRegressor

xgbm = XGBRegressor(random_state=123, max_depth=10)
xgbm.fit(X_train, y_train)

y_pred_xgbm = xgbm.predict(X_test)
rmse_xgbm = np.sqrt(mean_squared_error(y_test, y_pred_xgbm))
print("XGBR RMSE:", rmse_xgbm)

XGBR RMSE: 12.697554173579679


# Submission

In [13]:
preds1 = ensemble_model.predict(test)
preds2 = xgbm.predict(test)

preds = preds1*0.7 + preds2*0.3

ss['emission'] = preds
ss.to_csv('submission.csv', index=False)
ss.describe().T

,count,mean,std,min,25%,50%,75%,max
emission,24353.0,85.474894,148.637894,-2.360281,10.420847,47.838159,114.07662,3081.304272


In [14]:
ss

,ID_LAT_LON_YEAR_WEEK,emission
0,ID_-0.510_29.290_2022_00,3.740120
1,ID_-0.510_29.290_2022_01,4.073490
2,ID_-0.510_29.290_2022_02,4.139798
3,ID_-0.510_29.290_2022_03,4.214635
4,ID_-0.510_29.290_2022_04,4.348998
...,...,...
24348,ID_-3.299_30.301_2022_44,29.916625
24349,ID_-3.299_30.301_2022_45,30.371662
24350,ID_-3.299_30.301_2022_46,30.817705
24351,ID_-3.299_30.301_2022_47,30.999749
